In [ ]:
import requests
import json
from openai import AzureOpenAI
import logging

# Thêm logging configuration ở đầu file
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Thiết lập API key cho Azure OpenAI và Airtable
azure_api_key = ""  # Thay bằng API key của bạn
airtable_api_key = ""  # Thay bằng Airtable API key
base_id = "app7mcJMf9O8GQRal"  # Base ID của bạn
table_name = "tiktok_post"  # Table ID của bạn
view = "0_extract"  # View ID của bạn

azure_endpoint = "https://stepup.openai.azure.com"  # Thay bằng endpoint của bạn
api_version = "2024-08-01-preview"  # Phiên bản API
openai_model = "gpt-4o"  # Model OpenAI

# Thiết lập client Azure OpenAI
client = AzureOpenAI(
    azure_endpoint=azure_endpoint,
    api_key=azure_api_key,
    api_version=api_version
)

# Prompt hệ thống
system_prompt = """
Bạn là chuyên gia marketing, thành thạo phân tích video và social media \
Nhiệm vụ của bạn là khai thác insights từ nội dung các video và comment về đề tài học tiếng Anh\
Đọc và tìm ra những thông tin sau, trả kết quả dạng JSON\n\n

1. AI_filter: Kiểm tra xem nội dung này có đề cập tới việc học tiếng Anh hay không.\n
- english_learning: liên quan đến việc học tiếng Anh, hoặc giao tiếp tiếng Anh\n
- learning: việc học nói chung, không phải học tiếng Anh\n
- other: khác, không liên quan đến việc học\n\n

2. AI_purpose: Xác định mục đích của video này\n
- Promotional: video quảng cáo sản phẩm, nhằm nhắm đến chuyển đổi như giới thiệu sản phẩm, vào website...\n
- Mục đích khác nếu có \n\n

3. AI_product: \n
- Product: Xác định các sản phẩm/dịch vụ được nhắc tới trong video này nhằm mục đích quảng bá tăng chuyển đổi
- Solution: Tính năng, giải pháp, phương pháp học tiếng Anh được nhắc đến

4. AI_reverse: Phân tích nội dung video theo các chiều thông tin sau, trả về kết quả dạng bullet:\n
- Hook: câu hook mà người nói dùng để thu hút người nghe \n
- Message: Thông điệp chính mà người nói muốn truyền tải\n
- Audience: đối tượng mà người nói đang hướng tới hoặc thu hút chuyển đổi\n
- Insights: insight/vấn đề của người nghe mà người nói muốn chạm vào và giải quyết \n
- Outline: cấu trúc của video theo thứ tự thời gian này\n
- Technique: các kỹ thuật marketing mà người nói sử dụng trong việc làm content hoặc thiết kế video\n
- Research: các nghiên cứu khoa học được đề cập đến\n

5. AI_analysis: Đánh giá về video này dưới góc độ chuyên gia marketing\n

6. AI_other_insights: Ngoài những điều trên, bạn còn phát hiện ra điểm gì thú vị, độc đáo trong video này không?
"""

def fetch_airtable_data(base_id, table_name, view, api_key, offset=None):
    try:
        url = f"https://api.airtable.com/v0/{base_id}/{table_name}"
        headers = {
            "Authorization": f"Bearer {api_key}",
        }
        params = {
            "view": view
        }
        if offset:
            params["offset"] = offset
            
        logger.info(f"Fetching data from Airtable with params: {params}")
        response = requests.get(url, headers=headers, params=params)
        
        if response.status_code == 200:
            data = response.json()
            logger.info(f"Successfully fetched {len(data.get('records', []))} records")
            return data
        else:
            logger.error(f"Failed to fetch data. Status code: {response.status_code}")
            logger.error(f"Response: {response.text}")
            return None
    except Exception as e:
        logger.error(f"Error fetching data: {str(e)}")
        return None

def update_airtable_record(base_id, table_name, api_key, record_id, fields):
    url = f"https://api.airtable.com/v0/{base_id}/{table_name}/{record_id}"
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }
    data = {"fields": fields}
    response = requests.patch(url, headers=headers, json=data)
    if response.status_code != 200:
        logger.error(f"Failed to update record {record_id}. Status code: {response.status_code}, response: {response.text}")
    else:
        logger.info(f"Successfully updated record {record_id}")

def get_completion(prompt, record_id):
    try:
        logger.info(f"Sending request to OpenAI for record {record_id}")
        response = client.chat.completions.create(
            model=openai_model,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": prompt}
            ],
            temperature=0,
            top_p=1,
            presence_penalty=0,
            frequency_penalty=0,
            max_tokens=4000
        )
        logger.info(f"Received response from OpenAI for record {record_id}")
        return response
    except Exception as e:
        logger.error(f"Error in OpenAI API call for record {record_id}: {e}")
        return None

def test_openai_connection():
    try:
        logger.info("Kiểm tra kết nối Azure OpenAI")
        response = client.chat.completions.create(
            model=openai_model,
            messages=[{"role": "user", "content": "Hello"}],
            max_tokens=10
        )
        logger.info("Kết nối Azure OpenAI thành công")
        return True
    except Exception as e:
        logger.error(f"Lỗi kết nối Azure OpenAI: {str(e)}")
        return False

def format_reverse_to_bullets(data_dict):
    if not isinstance(data_dict, dict):
        return data_dict
        
    formatted_text = ""
    for key, value in data_dict.items():
        if isinstance(value, list):
            formatted_text += f"- {key}:\n"
            for item in value:
                formatted_text += f"  + {item}\n"
        else:
            formatted_text += f"- {key}: {value}\n"
    return formatted_text.strip()

def main():
    try:
        logger.info("Bắt đầu chạy chương trình")
        offset = None
        while True:
            logger.info("Đang fetch dữ liệu từ Airtable...")
            data = fetch_airtable_data(base_id, table_name, view, airtable_api_key, offset)

            if not data:
                logger.warning("Không lấy được dữ liệu từ Airtable")
                break

            records = data.get("records", [])
            logger.info(f"Số lượng records lấy được: {len(records)}")

            for record in records:
                record_id = record.get("id")
                logger.info(f"Đang xử lý record ID: {record_id}")
                
                fields = record.get("fields", {})
                title = fields.get("title", "")
                content_text = fields.get("desc", "")
                transcript = fields.get("transcript", "")
                comments = fields.get("comments_1", "")

                logger.info(f"Dữ liệu của record {record_id}:")
                logger.info(f"Title: {title[:50]}...")
                logger.info(f"Content: {content_text[:50]}...")
                logger.info(f"Script: {transcript[:50]}...")
                logger.info(f"Comments: {comments[:50]}...")

                if content_text:  # Chỉ kiểm tra content_text
                    logger.info(f"Đang tạo prompt cho record {record_id}")
                    full_content = f"""
                    Mô tả video từ người đăng: {content_text}
                    Transcript của video lấy từ audio: {transcript if transcript else 'Không có'}
                    """
                    
                    logger.info(f"Đang gọi OpenAI API cho record {record_id}")
                    response = get_completion(full_content, record_id)
                    
                    if response is None:
                        logger.warning(f"Không nhận được response từ OpenAI cho record {record_id}")
                        continue

                    try:
                        logger.info(f"Đang xử lý response cho record {record_id}")
                        ai_message_content = response.choices[0].message.content.strip()
                        logger.info(f"Response content: {ai_message_content[:100]}...")

                        # Loại bỏ markdown code block markers
                        if ai_message_content.startswith('```json'):
                            ai_message_content = ai_message_content[7:]
                        if ai_message_content.endswith('```'):
                            ai_message_content = ai_message_content[:-3]
                        
                        # Trim whitespace
                        ai_message_content = ai_message_content.strip()
                        
                        logger.info(f"Cleaned JSON content: {ai_message_content[:100]}...")
                        
                        # Parse JSON
                        logger.info("Đang parse JSON response")
                        parsed_json = json.loads(ai_message_content)
                        
                        # Chuẩn bị dữ liệu để update
                        logger.info("Đang chuẩn bị dữ liệu để update Airtable")
             
                        extracted_fields = {
                            "AI_filter": parsed_json.get("AI_filter", "no information"),
                            "AI_purpose": ", ".join(parsed_json.get("AI_purpose", [])) if isinstance(parsed_json.get("AI_purpose"), list) else parsed_json.get("AI_purpose", "no information"),
                            "AI_product": format_reverse_to_bullets(parsed_json.get("AI_product", {})),
                            "AI_reverse": format_reverse_to_bullets(parsed_json.get("AI_reverse", {})),
                            "AI_analysis": parsed_json.get("AI_analysis", "no information"),
                            "AI_other_insights": parsed_json.get("AI_other_insights", "no information")
                        }
                        
                        # Log extracted fields trước khi update
                        logger.info(f"Extracted fields: {json.dumps(extracted_fields, ensure_ascii=False)}")
                        
                        # Update Airtable
                        logger.info(f"Đang update record {record_id} vào Airtable")
                        update_airtable_record(base_id, table_name, airtable_api_key, record_id, extracted_fields)
                        logger.info(f"Đã update thành công record {record_id}")
                        
                    except json.JSONDecodeError as e:
                        logger.error(f"Lỗi parse JSON cho record {record_id}: {e}")
                        logger.error(f"JSON content causing error: {ai_message_content}")
                    except Exception as e:
                        logger.error(f"Lỗi xử lý response cho record {record_id}: {e}")
                else:
                    logger.warning(f"Thiếu content_text cho record {record_id}")

            offset = data.get("offset")
            if not offset:
                logger.info("Đã xử lý hết tất cả records")
                break

    except Exception as e:
        logger.error(f"Lỗi trong quá trình chạy: {str(e)}")
        raise e

if __name__ == "__main__":
    if test_openai_connection():
        main()
    else:
        logger.error("Không thể kết nối với Azure OpenAI, dừng chương trình")



2024-10-30 11:32:01,391 - INFO - Kiểm tra kết nối Azure OpenAI
2024-10-30 11:32:02,571 - INFO - HTTP Request: POST https://stepup.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-08-01-preview "HTTP/1.1 200 OK"
2024-10-30 11:32:02,577 - INFO - Kết nối Azure OpenAI thành công
2024-10-30 11:32:02,578 - INFO - Bắt đầu chạy chương trình
2024-10-30 11:32:02,586 - INFO - Đang fetch dữ liệu từ Airtable...
2024-10-30 11:32:02,588 - INFO - Fetching data from Airtable with params: {'view': '0_extract'}
2024-10-30 11:32:07,483 - INFO - Successfully fetched 100 records
2024-10-30 11:32:07,485 - INFO - Số lượng records lấy được: 100
2024-10-30 11:32:07,485 - INFO - Đang xử lý record ID: recK4MRLCmOkYYmzh
2024-10-30 11:32:07,486 - INFO - Dữ liệu của record recK4MRLCmOkYYmzh:
2024-10-30 11:32:07,486 - INFO - Title: ...
2024-10-30 11:32:07,487 - INFO - Content: Chỉ cần 1 kênh duy nhất có thể giúp bạn cải thiện ...
2024-10-30 11:32:07,487 - INFO - Script: [0:00:00]  Nếu như 

2024-10-30 11:32:18,686 - INFO - Sending request to OpenAI for record recqMYyJ3ZKgmkw69
2024-10-30 11:32:25,614 - INFO - HTTP Request: POST https://stepup.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-08-01-preview "HTTP/1.1 200 OK"
2024-10-30 11:32:25,619 - INFO - Received response from OpenAI for record recqMYyJ3ZKgmkw69
2024-10-30 11:32:25,620 - INFO - Đang xử lý response cho record recqMYyJ3ZKgmkw69
2024-10-30 11:32:25,621 - INFO - Response content: ```json
{
  "AI_filter": "english_learning",
  "AI_purpose": "Promotional",
  "AI_product": {
    "P...
2024-10-30 11:32:25,622 - INFO - Cleaned JSON content: {
  "AI_filter": "english_learning",
  "AI_purpose": "Promotional",
  "AI_product": {
    "Product":...
2024-10-30 11:32:25,623 - INFO - Đang parse JSON response
2024-10-30 11:32:25,625 - INFO - Đang chuẩn bị dữ liệu để update Airtable
2024-10-30 11:32:25,626 - INFO - Extracted fields: {"AI_filter": "english_learning", "AI_purpose": "Promotional", "A

2024-10-30 11:32:46,621 - INFO - Đã update thành công record rec1rNXurAK1ePQG2
2024-10-30 11:32:46,622 - INFO - Đang xử lý record ID: recAp2Rx6nqwmCVmf
2024-10-30 11:32:46,622 - INFO - Dữ liệu của record recAp2Rx6nqwmCVmf:
2024-10-30 11:32:46,623 - INFO - Title: ...
2024-10-30 11:32:46,624 - INFO - Content: Trả lời @Van pink  Rì viu cuốn sách luyện từ vựng ...
2024-10-30 11:32:46,624 - INFO - Script: [0:00:00]  có app nào luyện về từ vựng không?
[0:0...
2024-10-30 11:32:46,625 - INFO - Comments: [daisy.panh]
[Viết rõ tên web với ạ]...
2024-10-30 11:32:46,625 - INFO - Đang tạo prompt cho record recAp2Rx6nqwmCVmf
2024-10-30 11:32:46,626 - INFO - Đang gọi OpenAI API cho record recAp2Rx6nqwmCVmf
2024-10-30 11:32:46,626 - INFO - Sending request to OpenAI for record recAp2Rx6nqwmCVmf
2024-10-30 11:32:51,024 - INFO - HTTP Request: POST https://stepup.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-08-01-preview "HTTP/1.1 200 OK"
2024-10-30 11:32:51,027 - INFO - Re

2024-10-30 11:33:02,173 - INFO - Đang update record recVa08j0JgBBITHq vào Airtable
2024-10-30 11:33:03,607 - INFO - Successfully updated record recVa08j0JgBBITHq
2024-10-30 11:33:03,608 - INFO - Đã update thành công record recVa08j0JgBBITHq
2024-10-30 11:33:03,608 - INFO - Đang xử lý record ID: recRjfc3L8RiU7HYZ
2024-10-30 11:33:03,609 - INFO - Dữ liệu của record recRjfc3L8RiU7HYZ:
2024-10-30 11:33:03,610 - INFO - Title: ...
2024-10-30 11:33:03,610 - INFO - Content: Trả lời @Aries  Luyện tập roleplay hội thoại tình ...
2024-10-30 11:33:03,611 - INFO - Script: [0:00:00]  about our favorite sports and activitie...
2024-10-30 11:33:03,612 - INFO - Comments: [hactieuday]
[chị ưi em bên em đăng nhập mà hong h...
2024-10-30 11:33:03,612 - INFO - Đang tạo prompt cho record recRjfc3L8RiU7HYZ
2024-10-30 11:33:03,612 - INFO - Đang gọi OpenAI API cho record recRjfc3L8RiU7HYZ
2024-10-30 11:33:03,613 - INFO - Sending request to OpenAI for record recRjfc3L8RiU7HYZ
2024-10-30 11:33:10,212 - INFO - HT

2024-10-30 11:33:21,907 - INFO - Đang chuẩn bị dữ liệu để update Airtable
2024-10-30 11:33:21,908 - INFO - Extracted fields: {"AI_filter": "english_learning", "AI_purpose": "Promotional", "AI_product": "- Product: Pi - AI assistant\n- Solution: Improving English writing skills through daily chat", "AI_reverse": "- Hook: I'm so bored today\n- Message: Use the AI assistant Pi to improve your English writing skills through fun and engaging daily chats.\n- Audience: Individuals interested in improving their English writing skills, particularly those who enjoy chatting and learning online.\n- Insights: People often find traditional learning methods boring and are looking for more engaging ways to practice English.\n- Outline:\n  + Introduction with a relatable feeling of boredom\n  + Suggestion to use Pi for fun and engaging learning\n  + Description of Pi's user-friendly interface\n  + Encouragement to use Pi daily for short periods\n- Technique:\n  + Relatable opening to capture attention

...

2024-10-30 11:46:18,776 - INFO - Cleaned JSON content: {
  "AI_filter": "english_learning",
  "AI_purpose": "Promotional",
  "AI_product": {
    "Product":...
2024-10-30 11:46:18,777 - INFO - Đang parse JSON response
2024-10-30 11:46:18,778 - INFO - Đang chuẩn bị dữ liệu để update Airtable
2024-10-30 11:46:18,780 - INFO - Extracted fields: {"AI_filter": "english_learning", "AI_purpose": "Promotional", "AI_product": "- Product: Sách IELTS từ vựng và kỹ thuật đọc hiểu\n- Solution: Bộ video bài giảng song ngữ của cựu giám khảo IELTS", "AI_reverse": "- Hook: Giveaway tiếp này chúng mày ơi\n- Message: Tham gia giveaway để nhận sách IELTS và video bài giảng miễn phí\n- Audience: Người học tiếng Anh, đặc biệt là những người chuẩn bị thi IELTS\n- Insights: Người học cần tài liệu và hướng dẫn để cải thiện kỹ năng đọc hiểu IELTS\n- Outline:\n  + Giới thiệu giveaway và lý do tổ chức\n  + Thông tin về giải thưởng và cách tham gia\n  + Thể lệ và quy định của giveaway\n  + Thời gian công bố kết quả 

2024-10-30 11:46:31,843 - INFO - Đang update record recKipSYC0x1D5qda vào Airtable
2024-10-30 11:46:33,141 - INFO - Successfully updated record recKipSYC0x1D5qda
2024-10-30 11:46:33,143 - INFO - Đã update thành công record recKipSYC0x1D5qda
2024-10-30 11:46:33,144 - INFO - Đang xử lý record ID: rec2NXxXSeEYBgpky
2024-10-30 11:46:33,145 - INFO - Dữ liệu của record rec2NXxXSeEYBgpky:
2024-10-30 11:46:33,146 - INFO - Title: ...
2024-10-30 11:46:33,147 - INFO - Content: ...
2024-10-30 11:46:33,148 - INFO - Script: [0:00:00]  Làm ơn né ngay cái bộ cam ra nếu như mà...
2024-10-30 11:46:33,149 - INFO - Comments: [nickycater]
[tui cũng học des nhưng mà b2 thấy từ...
2024-10-30 11:46:33,150 - WARNING - Thiếu content_text cho record rec2NXxXSeEYBgpky
2024-10-30 11:46:33,150 - INFO - Đang xử lý record ID: recHajmE8a2yVITQL
2024-10-30 11:46:33,151 - INFO - Dữ liệu của record recHajmE8a2yVITQL:
2024-10-30 11:46:33,152 - INFO - Title: ...
2024-10-30 11:46:33,152 - INFO - Content: Tình yêu của elmmm 

2024-10-30 11:47:04,440 - INFO - Received response from OpenAI for record recKw6niYn9puxgNn
2024-10-30 11:47:04,442 - INFO - Đang xử lý response cho record recKw6niYn9puxgNn
2024-10-30 11:47:04,443 - INFO - Response content: ```json
{
  "AI_filter": "english_learning",
  "AI_purpose": "Educational",
  "AI_product": {
    "P...
2024-10-30 11:47:04,443 - INFO - Cleaned JSON content: {
  "AI_filter": "english_learning",
  "AI_purpose": "Educational",
  "AI_product": {
    "Product":...
2024-10-30 11:47:04,444 - INFO - Đang parse JSON response
2024-10-30 11:47:04,445 - INFO - Đang chuẩn bị dữ liệu để update Airtable
2024-10-30 11:47:04,446 - INFO - Extracted fields: {"AI_filter": "english_learning", "AI_purpose": "Educational", "AI_product": "- Product: Bộ sách học tiếng Anh ngắn gọn với phần tổng hợp từ vựng\n- Solution: Đọc sách tiếng Anh ngắn gọn, có phần tổng hợp từ vựng và nghĩa tiếng Việt", "AI_reverse": "- Hook: Chân thành khuyên chúng mày phải rút kinh nghiệm ngay, đừng có mà ngu n

2024-10-30 11:47:16,530 - INFO - Đang update record rec1Vgkl55sd3QCtR vào Airtable
2024-10-30 11:47:18,040 - INFO - Successfully updated record rec1Vgkl55sd3QCtR
2024-10-30 11:47:18,042 - INFO - Đã update thành công record rec1Vgkl55sd3QCtR
2024-10-30 11:47:18,043 - INFO - Đang xử lý record ID: recHVxqGXd8XYO0Lb
2024-10-30 11:47:18,044 - INFO - Dữ liệu của record recHVxqGXd8XYO0Lb:
2024-10-30 11:47:18,045 - INFO - Title: ...
2024-10-30 11:47:18,046 - INFO - Content: Sao lại không dạy được em ruột của mình nhỉ? 🥲 #tu...
2024-10-30 11:47:18,047 - INFO - Script: [0:00:00]  Chúng mình có hiểu cái cảm giác là dạy ...
2024-10-30 11:47:18,048 - INFO - Comments: [haru.com33]
[Em hiểu cảm giác của chị vì em chính...
2024-10-30 11:47:18,049 - INFO - Đang tạo prompt cho record recHVxqGXd8XYO0Lb
2024-10-30 11:47:18,049 - INFO - Đang gọi OpenAI API cho record recHVxqGXd8XYO0Lb
2024-10-30 11:47:18,050 - INFO - Sending request to OpenAI for record recHVxqGXd8XYO0Lb
2024-10-30 11:47:21,745 - INFO - HT

2024-10-30 11:47:35,252 - INFO - Đang update record recqL5dti34P83vuk vào Airtable
2024-10-30 11:47:37,146 - INFO - Successfully updated record recqL5dti34P83vuk
2024-10-30 11:47:37,149 - INFO - Đã update thành công record recqL5dti34P83vuk
2024-10-30 11:47:37,150 - INFO - Đang xử lý record ID: recxBzBi9XZ62KQqS
2024-10-30 11:47:37,151 - INFO - Dữ liệu của record recxBzBi9XZ62KQqS:
2024-10-30 11:47:37,152 - INFO - Title: ...
2024-10-30 11:47:37,153 - INFO - Content: Ai gấp thì tham khảo, kh gấp thì tự mò cũng chả vấ...
2024-10-30 11:47:37,153 - INFO - Script: [0:00:00]  Mình nghĩ là kỹ năng nào thì cũng tự họ...
2024-10-30 11:47:37,155 - INFO - Comments: [clevindang]
[học IELTS thì trăm bề đừng học trung...
2024-10-30 11:47:37,155 - INFO - Đang tạo prompt cho record recxBzBi9XZ62KQqS
2024-10-30 11:47:37,156 - INFO - Đang gọi OpenAI API cho record recxBzBi9XZ62KQqS
2024-10-30 11:47:37,157 - INFO - Sending request to OpenAI for record recxBzBi9XZ62KQqS
2024-10-30 11:47:41,727 - INFO - HT